In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


NOTE: Currently, the circuit is too huge, we would do it by requesting covariance and stdev and just calculate it on verifier side instead! Other circuits in this example can run well!

In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [12]:
scales = [5]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [13]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.correlation(x, y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [14]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 1) ------------->

+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error  | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.00537169 | 0.01074338   | 0.01074338 | 0         | 0.00537169     | 0.01074338       | 0.01074338    | 0             | 0.000057710102     | 0.0073635895       | 0.0073635895           |
+------------+--------------+------------+-----------+----------------+------------------+---------------+--------------

==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":1,"lookup_range":[-32398,7566],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":1250,"total_const_size":169,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-16,16]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709450442648}


In [15]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 6.017360210418701 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.71875
==== Generating Proof ====
proof:  {'instances': [['7469705c7b45875730aac31b009e8f9e9f9e57031bba148110877f78e4518c22', 'c1f946e3fd2b2229bfcc2dc9ff7f4dfc1fddad53635becb58d130bcae5504712', '0100000000000000000000000000000000000000000000000000000000000000', '1700000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x129fc9190be4bd89491477f1d610b0df57db6207db7622e3499b29b0526b7fde04ba713964b0f077421f98b36f8825b35bdf48aa966de7b13e525cb66435dcb51ff2fac415de8e4730b36682da9fa0fb4f6d7aea42d6931bbf9116554f9ea8f900019846562ac1c18a85a9d8520f2ee4ed1960b51a51e59cae1e439bc2c53a961bdf798b01322bc925a47a09944baee64ded964b88be194f7e12ea7af0fced5a11cd22e545c4ccb2e1359572e197c2c857f6b45899bfcb3aaaed6dca741126e9118e3b88e6bb3194e0e16dc54c9bb339f35f03fa65e735dc1a497b8794df2ac929901e945306c5739558e0e886a9b8cb31a020864b8bc1afee0e830abcb7c41028a0ffde9

In [17]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [0.71875]
